## Bibliotecas

In [ ]:
# Bibliotecas importadas

import pandas as pd # para visualização, manipulação e análise de dados
from google.colab import drive # para importar arquivos do Google Drive
drive.mount('/content/drive')

from sklearn.feature_selection import RFE # recursive feature elimination
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split # para separar o conjunto de dados em treinamento e teste para o modelo

import torch
from torch import tensor, nn, optim, from_numpy
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings # para desativar mensagens de advertência
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Funções para automatização

In [ ]:
def criar_dataframe(endereco, delimitador=','):
  """ Função que lê arquivo em .csv com a biblioteca pandas e o armazena na variável desejada.

      endereço: inserir string do local em que o arquivo está armazenado;
      delimitador: inserir string do delimitador deste arquivo .csv;
  """
  return pd.read_csv(endereco, delimiter=delimitador)

## Leitura e tratamento dos dados

In [ ]:
treino = criar_dataframe('/content/drive/MyDrive/Projeto - Sistema Anti-fraude/train.csv', ',')
teste = criar_dataframe('/content/drive/MyDrive/Projeto - Sistema Anti-fraude/test.csv', ',')

In [ ]:
# Tratamento do dataset de teste, para nomear corretamente as colunas
teste.columns = treino.columns

In [ ]:
print(f'O conjunto de teste possui {teste.shape[0]} instâncias a serem preditas.')

O conjunto de teste possui 704 instâncias a serem preditas.


## Divisão de atributos e classes

In [ ]:
# Removendo as colunas não normalizadas (Time e Amount)

treino.drop(columns=['Time', 'Amount'], inplace=True)
teste.drop(columns=['Time', 'Amount'], inplace=True)

In [ ]:
# Dividindo os conjuntos em atributos e classes
X_treino = treino.loc[:, treino.columns != 'Class']
y_treino = treino.loc[:, treino.columns == 'Class']

X_teste = teste.loc[:, treino.columns != 'Class']
y_teste = teste.loc[:, treino.columns == 'Class']

## Seleção de atributos (*Feature selection*)
Devemos testar Feature importance e/ou chi2? 

In [ ]:
selecao_atributos = {}
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=7, step=5, verbose=5)

In [ ]:
rfe_selector.fit(X_treino, y_treino)
rfe_support = rfe_selector.get_support()
rfe_feature_1 = X_treino.loc[:,rfe_support].columns.tolist()

Fitting estimator with 28 features.
Fitting estimator with 23 features.
Fitting estimator with 18 features.
Fitting estimator with 13 features.
Fitting estimator with 8 features.


In [ ]:
selecao_atributos['RFE'] = rfe_feature_1

In [ ]:
selecao_atributos

{'RFE': ['V4', 'V10', 'V13', 'V14', 'V21', 'V22', 'V27']}

Ao escolher sete atributos, o algoritmo **RFE** optou por: **V4, V10, V13, V14, V21, V22 e V27**.

Quero usar a lib [xverse](https://towardsdatascience.com/introducing-xverse-a-python-package-for-feature-selection-and-transformation-17193cdcd067), que testa todos os métodos de feature selection de uma vez. Podemos fazer gráficos comparando a análise de cada método, para visualizar as features mais citadas

## Conjunto de dados com os sete atributos selecionados

In [ ]:
# Substituição da variável X_treino, deixando apenas os atributos selecionados pelo RFE
X_treino = X_treino[rfe_feature_1]

# Substituição da variável X_teste, deixando apenas os atributos selecionados pelo RFE
X_teste = teste[rfe_feature_1]

In [ ]:
X_teste_array_classe = X_teste.copy()
X_teste_array_classe['Class'] = y_teste['Class']
X_teste_array_classe = X_teste_array_classe.to_numpy()

In [ ]:
# Criação da versão em numpy do nosso dataset de treino
X_treino_array = X_treino.to_numpy()

# Criação da versão em numpy do nosso dataset de treino
X_teste_array = X_teste.to_numpy()

## Balanceamento de classes 

In [ ]:
# Inserção da classe no 'X_treino', para que possamos balanceá-lo com base nas classes

X_treino_balanceado = X_treino.copy()
X_treino_balanceado['Class'] = y_treino['Class']

In [ ]:
# Redução da classe maioritária (não fraude) para 85.000 instâncias e aumento da classe minoritária (fraude) para 15.000 instâncias

nao_fraude = X_treino_balanceado[X_treino_balanceado['Class'] == 0]
fraude = X_treino_balanceado[X_treino_balanceado['Class'] == 1]

df_classe0 = nao_fraude.sample(85000, random_state=1)
df_classe1 = fraude.sample(15000, random_state=1, replace=True)

X_treino_balanceado = pd.concat([df_classe0, df_classe1], axis=0)

## Criação do modelo

In [ ]:
# Criação da versão em numpy do nosso dataset balanceado

X_treino_balanceado_array = X_treino_balanceado.to_numpy()

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, dataset, scale_data=True):
        
        xy = dataset.astype(np.float32)
        self.len = xy.shape[0]
        
        x_data = xy[:, 0:-1]
        y_data = xy[:, -1:]

        #x_data = MinMaxScaler().fit_transform(x_data)

        self.x_data = from_numpy(x_data) 
        self.y_data = from_numpy(y_data) 

  def __len__(self):
        return self.len
        
  def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

In [ ]:
class MLP(nn.Module):
    def __init__(self, atributos, neuronios):
        super().__init__()
        self.layers = nn.Sequential(
          nn.Linear(atributos, neuronios[0]),
          nn.ReLU(),
          nn.Linear(neuronios[0], neuronios[1]),
          nn.ReLU(),
          nn.Linear(neuronios[1], 1),
          nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

In [ ]:
treino_tensor = Dataset(X_treino_balanceado_array)
teste_tensor = Dataset(X_teste_array_classe)

# Hiperparâmetros
tamanho_lote = 100
epocas = 200
taxa_de_aprendizagem = 0.01

# Criação do modelo
train_loader = DataLoader(treino_tensor, batch_size=tamanho_lote, shuffle=True, num_workers=0)
modelo = MLP(atributos=7, neuronios=(20,10))
criterio = nn.BCELoss(reduction='mean')
otimizador = optim.Adam(modelo.parameters(), lr=taxa_de_aprendizagem)

In [ ]:
modelo(treino_tensor.x_data).max()

tensor(0.7677, grad_fn=<MaxBackward1>)

## Treinamento do modelo

In [ ]:
# Loop de treinamento
for epoca in range(epocas):
    loss_batch = 0
    for i, data in enumerate(train_loader):
        # obter as entradas
        inputs, labels = data

        # Propagação para frente
        y_pred = modelo(inputs)

        # Calcular e imprimir o loss
        perda = criterio(y_pred, labels)

        # Regularização L2 - Mudar pow(2.0) por abs() para a regularização L1
        l2_lambda = 0.001
        l2_norm = sum(p.pow(2.0).sum() for p in modelo.parameters())
        perda = perda + l2_lambda * l2_norm

        loss_batch += perda

        # Backpropagation e atualização dos pesos
        otimizador.zero_grad()
        perda.backward()
        otimizador.step()

    if (epoca+1)%10 == 0 or epoca == 0:
        with torch.no_grad():
            # obter as entradas
            inputs_test, y_real = teste_tensor.x_data, teste_tensor.y_data
            
            # Calcular a precisão para o treino
            inputs_train = treino_tensor.x_data
            y_pred_train = modelo(inputs_train)
            f1_treino = f1_score(treino_tensor.y_data, torch.round(y_pred_train) )

            # Fazer a previsão do conjunto de teste com o modelo treinado até agora
            y_pred_test = modelo(inputs_test)

            # Métrica de precisão
            f1_teste = f1_score(y_real, torch.round(y_pred_test) )
            print(f'época: {epoca+1} | perda: {loss_batch:.2e} | F1-Score (treino): {100*f1_treino:.2f}% | F1-Score (teste): {100*f1_teste:.2f}%')

época: 1 | perda: 1.12e+02 | F1-Score (treino): 91.04% | F1-Score (teste): 92.23%
época: 10 | perda: 1.01e+02 | F1-Score (treino): 91.45% | F1-Score (teste): 93.62%
época: 20 | perda: 1.01e+02 | F1-Score (treino): 91.17% | F1-Score (teste): 91.75%
época: 30 | perda: 1.01e+02 | F1-Score (treino): 91.15% | F1-Score (teste): 91.28%
época: 40 | perda: 1.01e+02 | F1-Score (treino): 91.44% | F1-Score (teste): 92.15%
época: 50 | perda: 1.01e+02 | F1-Score (treino): 91.19% | F1-Score (teste): 91.28%
época: 60 | perda: 1.01e+02 | F1-Score (treino): 91.04% | F1-Score (teste): 91.58%
época: 70 | perda: 1.01e+02 | F1-Score (treino): 91.16% | F1-Score (teste): 91.75%
época: 80 | perda: 1.00e+02 | F1-Score (treino): 91.13% | F1-Score (teste): 91.67%
época: 90 | perda: 1.01e+02 | F1-Score (treino): 91.28% | F1-Score (teste): 93.12%
época: 100 | perda: 1.00e+02 | F1-Score (treino): 91.29% | F1-Score (teste): 92.63%
época: 110 | perda: 1.00e+02 | F1-Score (treino): 90.80% | F1-Score (teste): 93.62%
épo

## Salvamento do modelo

In [ ]:
save_path = './modelo_fraude'
torch.save(modelo.state_dict(), save_path)

## Verificação dos resultados

In [ ]:
previsao = torch.round(y_pred_test)

In [ ]:
y_previsto_array = previsao.numpy().astype(int)

In [ ]:
teste_modelo = y_teste.copy()
teste_modelo['Real'] = teste_modelo['Class']
teste_modelo.drop(columns=['Class'], inplace=True)
teste_modelo['Previsão'] = y_previsto_array
teste_modelo

,Real,Previsão
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
699,0,0
700,0,0
701,0,0
702,0,0
